<a href="https://colab.research.google.com/github/sailTo/essay/blob/master/V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import SGDClassifier, LogisticRegression

dfoff = pd.read_csv('/content/drive/Shareddrives/ubox/ccf_offline_stage1_train.csv')
dftest = pd.read_csv('/content/drive/Shareddrives/ubox/ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('/content/drive/Shareddrives/ubox/ccf_online_stage1_train.csv')
print('All data are read.')

All data are read.


In [ ]:
def float_to_datetime(row):
    if row != 'nan': return date(int(row[0:4]), int(row[4:6]), int(row[6:8]))
    else: return np.nan

dfoff['Date_received'] = dfoff['Date_received'].astype(str).apply(float_to_datetime)
dftest['Date_received'] = dftest['Date_received'].astype(str).apply(float_to_datetime)
dfoff['Date'] = dfoff['Date'].astype(str).apply(float_to_datetime)


In [ ]:
def get_offline_features(X, offline):
    # X = X[:1000]

    print(len(X), len(X.columns))

    coupon_id_notnull = offline[offline.Coupon_id.notnull()]

    temp = coupon_id_notnull
    coupon_consume = temp[temp.Date.notnull()]
    coupon_no_consume = temp[temp.Date.isnull()]

    user_coupon_consume = coupon_consume.groupby('User_id')

    '''user features'''

    # 用户优惠券消费次数
    temp = user_coupon_consume.size().reset_index(name='u2')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.u2 = X.u2.fillna(0)

    # 用户优惠券不消费次数
    temp = coupon_no_consume.groupby('User_id').size().reset_index(name='u3')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券消费与不消费次数比值
    X['u19'] = X.u2 / X.u3

    # 用户领取优惠券次数
    X['u1'] = X.u2.fillna(0) + X.u3.fillna(0)

    # 用户优惠券使用率
    X['u4'] = X.u2 / X.u1
    X.u4 = X.u4.fillna(sum(X.u4.fillna(0))/sum(X.u4.notnull()))

    # 用户普通消费次数
    date_received_isnull_date_notnull = offline[offline.Date_received.isnull() & offline.Date.notnull()]

    temp = date_received_isnull_date_notnull.groupby('User_id').size().reset_index(name='u5')
    X = pd.merge(X, temp, how='left', on='User_id')
    print(X.columns)
    X.u5 = X.u5.fillna(sum(X.u5.fillna(0))/sum(X.u5.notnull()))

    # 用户优惠券消费和普通消费次数
    X['u25'] = X.u2 + X.u5

    # 用户使用优惠券消费占比
    X['u20'] = X.u2 / X.u25
    X.u20 = X.u20.fillna(sum(X.u20.fillna(0))/sum(X.u20.notnull()))

    # 用户领取所有不同优惠券数量
    date_received_notnull = offline[offline.Date_received.notnull()]
    temp = date_received_notnull
    temp = temp.groupby(['User_id', 'Coupon_id']).size().reset_index(name='u47')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id'])

    # 核销优惠券用户-商家平均距离 fill_avg
    temp = offline[offline.Coupon_id.notnull() & offline.Date.notnull() & offline.Distance.notnull()]
    temp = temp.groupby('User_id').Distance
    temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='User_id')
    temp['u35'] = temp.y / temp.x
    temp = temp[['User_id', 'u35']]
    X = pd.merge(X, temp, how='left', on='User_id')
    X.u35=X.u35.fillna(sum(X.u35.fillna(0))/sum(X.u35.notnull()))

    # 用户核销优惠券中的最小用户-商家距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('User_id').Distance.min().reset_index(name='u36')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.u36=X.u36.fillna(sum(X.u36.fillna(0))/sum(X.u36.notnull()))

    # 用户核销优惠券中的最大用户-商家距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('User_id').Distance.max().reset_index(name='u37')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.u37=X.u37.fillna(sum(X.u37.fillna(0))/sum(X.u37.notnull()))

    # 优惠券类型
    discount_types = [
        '0.2', '0.5', '0.6', '0.7', '0.75', '0.8', '0.85', '0.9', '0.95', '30:20', '50:30', '10:5',
        '20:10', '100:50', '200:100', '50:20', '30:10', '150:50', '100:30', '20:5', '200:50', '5:1',
        '50:10', '100:20', '150:30', '30:5', '300:50', '200:30', '150:20', '10:1', '50:5', '100:10',
        '200:20', '300:30', '150:10', '300:20', '500:30', '20:1', '100:5', '200:10', '30:1', '150:5',
        '300:10', '200:5', '50:1', '100:1',
    ]
    X['discount_type'] = -1
    for k, v in enumerate(discount_types):
        X.loc[X.Discount_rate == v, 'discount_type'] = k

    '''offline merchant features'''

    # 商家销售次数
    temp = offline[offline.Date.notnull()].groupby('Merchant_id').size().reset_index(name='m0')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销次数
    temp = coupon_consume.groupby('Merchant_id').size().reset_index(name='m1')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家普通销售次数
    X['m2'] = X.m0.fillna(0) - X.m1.fillna(0)

    # 商家优惠券被领取次数
    temp = date_received_notnull.groupby('Merchant_id').size().reset_index(name='m3')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销率
    X['m4'] = X.m1 / X.m3
    X.m4=X.m4.fillna(sum(X.m4.fillna(0))/sum(X.m4.notnull()))


    # 商家在数据集中出现的次数
    temp = offline.groupby('Merchant_id').size().reset_index(name='m16')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家被核销优惠券的平均时间
    temp = pd.merge(coupon_consume, coupon_consume.groupby('Merchant_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('Merchant_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('Merchant_id').size().reset_index(name='_len'))
    temp['m20'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('Merchant_id', inplace=True)
    X = pd.merge(X, temp[['Merchant_id', 'm20']], how='left', on='Merchant_id')
    X.m20 = X.m20.fillna(sum(X.m20.fillna(0))/sum(X.m20.notnull()))

    # 商家被核销优惠券中的用户平均距离
    temp = coupon_consume[coupon_consume.Distance.notnull()].groupby('Merchant_id').Distance
    temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='Merchant_id')
    temp['m21'] = temp.y / temp.x
    temp = temp[['Merchant_id', 'm21']]
    X = pd.merge(X, temp, how='left', on='Merchant_id')
    X.m21=X.m21.fillna(sum(X.m21.fillna(0))/sum(X.m21.notnull()))

    """offline coupon features"""

    # 每种优惠券领取次数
    temp = coupon_id_notnull.groupby('Coupon_id').size().reset_index(name='c1')
    X = pd.merge(X, temp, how='left', on='Coupon_id')
    X.c1=X.c1.fillna(0)

    # 每种优惠券使用次数
    temp = coupon_consume.groupby('Coupon_id').size().reset_index(name='c2')
    X = pd.merge(X, temp, how='left', on='Coupon_id')
    X.c2 = X.c2.fillna(0)

    # 优惠券使用率
    X['c3'] = X.c2 / X.c1
    X.drop(['c1','c2'],axis=1)
    X.c3 = X.c3.fillna(sum(X.c3.fillna(0))/sum(X.c3.notnull()))

    # 优惠券类型(直接优惠为0, 满减为1)
    X['c6'] = 0 
    X.loc[X.Discount_rate.str.contains(':') == True, 'c6'] = 1

    # 每种优惠券核销的平均时间
    temp = pd.merge(coupon_consume, coupon_consume.groupby('Coupon_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('Coupon_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('Coupon_id').size().reset_index(name='_len'))
    temp['c12'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('Coupon_id', inplace=True)
    X = pd.merge(X, temp[['Coupon_id', 'c12']], how='left', on='Coupon_id')
    X.c12=X.c12.fillna(sum(X.c12.fillna(0))/sum(X.c12.notnull()))

    '''user merchant feature'''

    # 用户领取商家的优惠券次数
    temp = coupon_id_notnull
    temp = temp.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um1')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])
    X.um1 = X.um1.fillna(0)

    # 用户领取商家的优惠券后核销次数
    temp = coupon_consume.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um3')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])
    X.um3 = X.um3.fillna(0)

    # 用户领取商家的优惠券后核销率
    X['um4'] = X.um3 / X.um1
    X.um4=X.um4.fillna(sum(X.um4.fillna(0))/sum(X.um4.notnull()))
    
    '''other feature'''

    # 用户领取优惠券次数
    temp = X.groupby('User_id').size().reset_index(name='o1')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.o1 = X.o1.fillna(sum(X.o1.fillna(0))/sum(X.o1.notnull()))
    
    print(len(X), len(X.columns))

    return X


def get_online_features(X, online):
    # temp = online[online.Coupon_id == online.Coupon_id]
    # coupon_consume = temp[temp.Date == temp.Date]
    # coupon_no_consume = temp[temp.Date != temp.Date]

    # 用户操作次数
    temp = online.groupby('User_id').size().reset_index(name='on_u1')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u1 = X.on_u1.fillna(sum(X.u37.fillna(0))/sum(X.u37.notnull()))

    # 用户点击次数
    temp = online[online.Action == 0].groupby('User_id').size().reset_index(name='on_u2')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u2 = X.on_u2.fillna(0)

    # 用户点击率
    X['on_u3'] = X.on_u2 / X.on_u1
    X.on_u3 = X.on_u3.fillna(0)

    # 用户购买次数
    temp = online[online.Action == 1].groupby('User_id').size().reset_index(name='on_u4')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u4 = X.on_u4.fillna(0)

    # 用户线上购买率
    X['on_u5'] = X.on_u4 / X.on_u1
    X.on_u5 = X.on_u5.fillna(0)

    # 用户领取优惠券次数
    temp = online[online.Date_received.notnull()].groupby('User_id').size().reset_index(name='on_u6')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u6 = X.on_u6.fillna(0)

    # 用户优惠券领取率
    X['on_u7'] = X.on_u6 / X.on_u1
    X.on_u7 = X.on_u7.fillna(0)

    # 用户优惠券消费次数
    temp = online[online.Date_received.notnull() & online.Date.notnull()]
    temp = temp.groupby('User_id').size().reset_index(name='on_u9')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u9 = X.on_u9.fillna(0)
    
    # 用户优惠券核销率
    X['on_u10'] = X.on_u9 / X.on_u6
    X = X.drop(['on_u1','on_u2','on_u4','on_u6','on_u9'],axis=1)
    X.on_u10 = X.on_u10.fillna(sum(X.on_u10.fillna(0))/sum(X.on_u10.notnull()))
    
    print(len(X), len(X.columns))
    print('----------')

    return X

In [ ]:
dftest = get_offline_features(dftest,dfoff)
dftest = get_online_features(dftest,dfon)

113640 6
Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'u2', 'u3', 'u19', 'u1', 'u4', 'u5'],
      dtype='object')
113640 36
113640 40
----------


In [ ]:
def getWeekday(row):
    if type(row)==type(0.0):
        return np.nan
    else:
        return row.weekday() + 1

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
dftest['weekday_type'] = dftest['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )

# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
tmpdf = pd.get_dummies(dfoff['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf


dfoff = get_offline_features(dfoff,dfoff)

# change discount type to one-hot encoding 
discounts = ['discount_' + str(i) for i in range(1,47)]
tmpdf = pd.get_dummies(dfoff['discount_type'].replace('nan', np.nan))
tmpdf.columns = discounts
dfoff[discounts] = tmpdf

tmpdf = pd.get_dummies(dftest['discount_type'].replace('nan', np.nan))
tmpdf[0] = np.zeros(tmpdf.shape[0])
tmpdf[4] = np.zeros(tmpdf.shape[0])
tmpdf[10] = np.zeros(tmpdf.shape[0])
tmpdf[42] = np.zeros(tmpdf.shape[0])
tmpdf = tmpdf[sorted(tmpdf.columns)]
tmpdf.columns = discounts
dftest[discounts] = tmpdf

def label(row):
    if pd.isnull(row['Date_received']):
        return -1
    if pd.notnull(row['Date']):
        td = row['Date'] - row['Date_received']
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
dfoff['label'] = dfoff.apply(label, axis = 1)
print("end")

1754884 16
Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'weekday', 'weekday_type', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'weekday_7', 'u2', 'u3', 'u19', 'u1', 'u4', 'u5'],
      dtype='object')
1754884 46
end


In [ ]:
# data split
print("-----data split------")
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < date(2016,5,16))].copy()
valid = df[(df['Date_received'] >= date(2016,5,16)) & (df['Date_received'] <= date(2016,6,15))].copy()
print("end")

-----data split------
end


In [ ]:
# for i in dfoff.columns:
#     if sum(dfoff[i].isnull())!=0: print(i)
dfoff.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'weekday', 'weekday_type', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'weekday_7', 'u2', 'u3', 'u19', 'u1', 'u4', 'u5', 'u25', 'u20', 'u47',
       'u35', 'u36', 'u37', 'discount_type', 'm0', 'm1', 'm2', 'm3', 'm4',
       'm16', 'm20', 'm21', 'c1', 'c2', 'c3', 'c6', 'c12', 'um1', 'um3', 'um4',
       'o1', 'discount_1', 'discount_2', 'discount_3', 'discount_4',
       'discount_5', 'discount_6', 'discount_7', 'discount_8', 'discount_9',
       'discount_10', 'discount_11', 'discount_12', 'discount_13',
       'discount_14', 'discount_15', 'discount_16', 'discount_17',
       'discount_18', 'discount_19', 'discount_20', 'discount_21',
       'discount_22', 'discount_23', 'discount_24', 'discount_25',
       'discount_26', 'discount_27', 'discount_28', 'discount_29',
       'discount_30', 'discount_31', 'discount_32', 'discount_33',


In [ ]:
# feature
original_feature = ['u4', 'u20', 'u35', 'u36', 'u37',# onehot
        'm4', 'm20', 'm21', 'c3', 'c6', 'c12', 'um4', 'o1', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'weekday_7', 'discount_1', 'discount_2', 'discount_3', 'discount_4',
       'discount_5', 'discount_6', 'discount_7', 'discount_8', 'discount_9',
       'discount_10', 'discount_11', 'discount_12', 'discount_13',
       'discount_14', 'discount_15', 'discount_16', 'discount_17',
       'discount_18', 'discount_19', 'discount_20', 'discount_21',
       'discount_22', 'discount_23', 'discount_24', 'discount_25',
       'discount_26', 'discount_27', 'discount_28', 'discount_29',
       'discount_30', 'discount_31', 'discount_32', 'discount_33',
       'discount_34', 'discount_35', 'discount_36', 'discount_37',
       'discount_38', 'discount_39', 'discount_40', 'discount_41',
       'discount_42', 'discount_43', 'discount_44', 'discount_45',
       'discount_46']# remove 'Distance' temporarily
train = train.fillna(-1)
print("----train-----")
model = SGDClassifier(#lambda:
    loss='log',
    penalty='elasticnet',
    fit_intercept=True,
    max_iter=100,
    shuffle=True,
    alpha = 0.01,
    l1_ratio = 0.01,
    n_jobs=1,
    class_weight=None
)
model.fit(train[original_feature], train['label'])

----train-----


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.01, learning_rate='optimal', loss='log', max_iter=100,
              n_iter_no_change=5, n_jobs=1, penalty='elasticnet', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
# test prediction for submission
test = dftest.fillna(-1)
y_test_pred = model.predict_proba(test[original_feature])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
def date_to_str(date):
    return date.strftime('%Y%m%d')
dftest1.Date_received = dftest1.Date_received.apply(date_to_str)
dftest1['label'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head()

,User_id,Coupon_id,Date_received,label
0,4129537,9983,20160712,0.025095
1,6949378,3429,20160706,0.148146
2,2166529,6928,20160727,0.019970
3,2166529,1808,20160727,0.025548
4,6172162,6500,20160708,0.037940


In [ ]:
a = np.sum(dfoff[original_feature].isnull())
a[a!=0]

Series([], dtype: int64)

In [ ]:
# dftest1[dftest1.label>0.5]
import statsmodels.api as sm
model = sm.tsa.ARIMA(dfoff.label,order=(1,1,1),exog=dfoff[original_feature])
res = model.fit()
print(res.summary())